In [161]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
df = pd.read_csv('without_missing.csv')

In [3]:
df.head()

,Unnamed: 0,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     1460 non-null   int64  
 1   Id             1460 non-null   int64  
 2   MSSubClass     1460 non-null   int64  
 3   MSZoning       1460 non-null   object 
 4   LotFrontage    1460 non-null   float64
 5   LotArea        1460 non-null   int64  
 6   Street         1460 non-null   object 
 7   Alley          91 non-null     object 
 8   LotShape       1460 non-null   object 
 9   LandContour    1460 non-null   object 
 10  Utilities      1460 non-null   object 
 11  LotConfig      1460 non-null   object 
 12  LandSlope      1460 non-null   object 
 13  Neighborhood   1460 non-null   object 
 14  Condition1     1460 non-null   object 
 15  Condition2     1460 non-null   object 
 16  BldgType       1460 non-null   object 
 17  HouseStyle     1460 non-null   object 
 18  OverallQ

In [5]:
df = df.drop(['GarageYrBlt', 'Unnamed: 0'], axis = 1)

In [6]:
df = df.fillna('None')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          1460 non-null   object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [8]:
X = df.drop('SalePrice', axis = 1)

In [9]:
y = df['SalePrice']

In [10]:
X = pd.get_dummies(X, drop_first=True, dtype=int)

In [11]:
X.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,0,1,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,0,1,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,0,1,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,0,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,0,1,0,0,0,1,0


In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [14]:
scaled_X = scaler.fit_transform(X)

In [15]:
from sklearn.decomposition import PCA

In [23]:
arr = []
for n in range(130, 170):
    pca_model = PCA(n_components=n)
    pca_model.fit(scaled_X)
    arr.append(np.sum(pca_model.explained_variance_ratio_))

plt.figure(figsize=(16, 16), dpi = 800)
plt.grid(True)
plt.plot(range(130, 170), arr)

In [16]:
pca_model = PCA(n_components=139)

In [17]:
pca_X = pca_model.fit_transform(scaled_X)

In [18]:
pca_X

array([[-4.32465591,  1.63196294, -0.25013858, ...,  0.08575557,
        -0.22443697,  0.03218518],
       [-0.0423353 , -2.91811178,  0.10256416, ..., -2.13297258,
         0.73677524,  0.08315002],
       [-5.20397237,  1.01634597,  0.18660996, ...,  0.33715195,
        -0.24316117, -0.24820087],
       ...,
       [-0.74217752,  0.40470917, -0.94188975, ..., -1.76804172,
         0.20762222,  2.23038939],
       [ 2.40683932, -3.33984432, -0.02459218, ..., -1.15463952,
        -0.28800996,  0.39132237],
       [ 1.26833061, -3.47291136, -0.04879338, ...,  0.48685064,
         0.33778779, -0.52085985]])

In [20]:
X = torch.tensor(pca_X).type(torch.float)

In [21]:
y = np.array(y)

In [22]:
y = torch.tensor(y).type(torch.float)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [65]:
model = nn.Sequential(
    nn.Linear(139, 20),
    nn.Linear(20, 1)
)

In [66]:
model

Sequential(
  (0): Linear(in_features=139, out_features=20, bias=True)
  (1): Linear(in_features=20, out_features=1, bias=True)
)

In [88]:
loss_fn = nn.MSELoss()

In [89]:
optim = torch.optim.Adam(params = model.parameters(), lr = 0.01)

In [90]:
torch.manual_seed(101)

epochs = 500

for epoch in range(epochs):
    model.train()
    y_pred = model(X_train).squeeze()
    loss = loss_fn(y_pred, y_train)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if epoch % 10 == 0:
        print(f'Epoch:{epoch}   |   Loss:{loss:.4f}')

Epoch:0   |   Loss:786391360.0000
Epoch:10   |   Loss:762064512.0000
Epoch:20   |   Loss:741606848.0000
Epoch:30   |   Loss:724675008.0000
Epoch:40   |   Loss:710707264.0000
Epoch:50   |   Loss:699158592.0000
Epoch:60   |   Loss:689604928.0000
Epoch:70   |   Loss:681702080.0000
Epoch:80   |   Loss:675165248.0000
Epoch:90   |   Loss:669759232.0000
Epoch:100   |   Loss:665287360.0000
Epoch:110   |   Loss:661585024.0000
Epoch:120   |   Loss:658516800.0000
Epoch:130   |   Loss:655971008.0000
Epoch:140   |   Loss:653856448.0000
Epoch:150   |   Loss:652097408.0000
Epoch:160   |   Loss:650632192.0000
Epoch:170   |   Loss:649409728.0000
Epoch:180   |   Loss:648387904.0000
Epoch:190   |   Loss:647531968.0000
Epoch:200   |   Loss:646813312.0000
Epoch:210   |   Loss:646208384.0000
Epoch:220   |   Loss:645697792.0000
Epoch:230   |   Loss:645265408.0000
Epoch:240   |   Loss:644898240.0000
Epoch:250   |   Loss:644585280.0000
Epoch:260   |   Loss:644317696.0000
Epoch:270   |   Loss:644088064.0000
Epo

In [42]:
from sklearn.metrics import mean_absolute_percentage_error

In [91]:
y_pred = model(X_test)
mean_absolute_percentage_error(y_test.detach().numpy(), y_pred.detach().numpy())

0.14460373

In [86]:
from sklearn.metrics import mean_squared_error

In [92]:
mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy())

2291445500.0

label encoder

In [95]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [116]:
X = df.drop('SalePrice', axis = 1)

In [117]:
X['LotFrontage'].value_counts(ascending=True)

LotFrontage
46.000000       1
141.000000      1
153.000000      1
150.000000      1
111.000000      1
             ... 
50.000000      57
80.000000      69
70.000000      70
60.000000     143
70.049958     259
Name: count, Length: 111, dtype: int64

In [118]:
y = df['SalePrice']

In [119]:
from sklearn.preprocessing import LabelEncoder

In [121]:
object_columns = X.select_dtypes(include=['object']).columns

In [122]:
for col in object_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])

In [123]:
X.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,1,1,3,3,0,4,...,0,0,3,4,1,0,2,2008,8,4
1,20,3,80.0,9600,1,1,3,3,0,2,...,0,0,3,4,1,0,5,2007,8,4
2,60,3,68.0,11250,1,1,0,3,0,4,...,0,0,3,4,1,0,9,2008,8,4
3,70,3,60.0,9550,1,1,0,3,0,0,...,0,0,3,4,1,0,2,2006,8,0
4,60,3,84.0,14260,1,1,0,3,0,2,...,0,0,3,4,1,0,12,2008,8,4


In [124]:
from sklearn.preprocessing import StandardScaler

In [125]:
scaler = StandardScaler()

In [126]:
scaled_X = scaler.fit_transform(X)

In [136]:
X = torch.tensor(scaled_X).type(torch.float)

In [137]:
y = y.to_numpy()

In [138]:
y = torch.tensor(y).type(torch.float)

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [177]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.layer_1 = nn.Linear(75, 10)
        self.layer_2 = nn.Linear(10, 1)

    def forward(self, X):
        X = F.relu(self.layer_1(X))
        X = F.relu(self.layer_2(X))
        return F.relu(X)

In [178]:
model = model()
model

model(
  (layer_1): Linear(in_features=75, out_features=10, bias=True)
  (layer_2): Linear(in_features=10, out_features=1, bias=True)
)

In [181]:
optim = torch.optim.Adam(params= model.parameters(), lr = 0.1)

In [182]:
loss_fn = nn.HuberLoss()

In [183]:
torch.manual_seed(101)

epochs = 10000

for epoch in range(epochs):
    model.train()
    optim.zero_grad()
    y_pred = model(X_train).squeeze()
    loss = loss_fn(y_pred, y_train)
    loss.backward()
    optim.step()
    if epoch % 10 == 0:
        print(f'Epoch:{epoch}   |   Loss:{loss:.4f}')

Epoch:0   |   Loss:181356.4688
Epoch:10   |   Loss:181304.0312
Epoch:20   |   Loss:181063.7969
Epoch:30   |   Loss:180545.0938
Epoch:40   |   Loss:179673.2188
Epoch:50   |   Loss:178391.4062
Epoch:60   |   Loss:176658.7656
Epoch:70   |   Loss:174448.9844
Epoch:80   |   Loss:171745.0156
Epoch:90   |   Loss:168541.0469
Epoch:100   |   Loss:164867.4375
Epoch:110   |   Loss:160710.5938
Epoch:120   |   Loss:156105.8281
Epoch:130   |   Loss:151138.5000
Epoch:140   |   Loss:146099.0625
Epoch:150   |   Loss:141072.5469
Epoch:160   |   Loss:136017.9375
Epoch:170   |   Loss:131016.6562
Epoch:180   |   Loss:126020.1797
Epoch:190   |   Loss:120987.5312
Epoch:200   |   Loss:115877.5625
Epoch:210   |   Loss:110791.2031
Epoch:220   |   Loss:105732.0156
Epoch:230   |   Loss:100599.7344
Epoch:240   |   Loss:95383.7109
Epoch:250   |   Loss:90035.7500
Epoch:260   |   Loss:84690.5391
Epoch:270   |   Loss:79391.4688
Epoch:280   |   Loss:74212.7266
Epoch:290   |   Loss:69125.5469
Epoch:300   |   Loss:64589.

In [184]:
y_pred = model(X_test)
mean_absolute_percentage_error(y_test.detach().numpy(), y_pred.detach().numpy())

0.16355895

In [169]:
from sklearn.metrics import mean_squared_error

In [185]:
mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy())

2984317700.0